# Model selection

## Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler


#NN parts
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import scipy as sp

## Load data

In [3]:
data = pd.read_csv('../../data/processed/processed_data.csv')
data.head()

,file,sexo,edad,peso,talla,per_brazo_rel,per_brazo_ten,per_antebrazo,per_torax,per_cintura,per_cadera,per_muslo_max,per_muslo_medial,per_pantorrilla,masa_piel,masa_adiposa,masa_muscular,masa_residual,masa_osea
0,ANTROPOMETRIA (24).xls,2,32.783025,75.8,160.0,31.8,33.0,26.5,98.8,80.3,98.5,62.5,55.7,39.5,3.750202,24.924776,32.163791,8.108962,9.331517
1,ANTROPOMODELO 28-4-2021.xls,1,20.240931,66.7,165.0,30.6,32.8,26.0,97.0,75.9,92.4,53.7,47.0,33.2,3.585589,17.189072,32.433673,7.148282,7.474786
2,ANTROPOMODELO 3-2-2021 (2).xls,1,25.092402,67.0,176.0,26.8,27.4,24.3,86.3,76.0,97.0,53.7,47.5,35.9,3.764521,24.290302,25.310796,6.384089,9.142289
3,ANTROPOMODELO COMUN (11).xls,2,24.605065,52.2,154.0,23.1,23.3,21.0,76.6,62.2,92.8,53.0,45.0,34.1,3.112940,16.307029,18.519120,4.430181,5.948666
4,ANTROPOMETRIA (18).xls,1,28.741958,101.0,162.5,40.7,43.8,32.8,112.5,99.5,111.4,68.5,62.8,42.0,4.229962,39.435706,41.820398,11.560786,10.609290


## add features

In [4]:
data = pd.concat([data,pd.get_dummies(data['sexo'],prefix='sexo')],axis=1)
data.drop(['sexo','file'],axis=1, inplace=True)
data


,edad,peso,talla,per_brazo_rel,per_brazo_ten,per_antebrazo,per_torax,per_cintura,per_cadera,per_muslo_max,per_muslo_medial,per_pantorrilla,masa_piel,masa_adiposa,masa_muscular,masa_residual,masa_osea,sexo_1,sexo_2
0,32.783025,75.8,160.0,31.8,33.0,26.5,98.8,80.3,98.5,62.5,55.7,39.5,3.750202,24.924776,32.163791,8.108962,9.331517,0,1
1,20.240931,66.7,165.0,30.6,32.8,26.0,97.0,75.9,92.4,53.7,47.0,33.2,3.585589,17.189072,32.433673,7.148282,7.474786,1,0
2,25.092402,67.0,176.0,26.8,27.4,24.3,86.3,76.0,97.0,53.7,47.5,35.9,3.764521,24.290302,25.310796,6.384089,9.142289,1,0
3,24.605065,52.2,154.0,23.1,23.3,21.0,76.6,62.2,92.8,53.0,45.0,34.1,3.112940,16.307029,18.519120,4.430181,5.948666,0,1
4,28.741958,101.0,162.5,40.7,43.8,32.8,112.5,99.5,111.4,68.5,62.8,42.0,4.229962,39.435706,41.820398,11.560786,10.609290,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,23.444216,91.0,167.0,34.0,35.0,27.0,106.0,85.5,112.0,71.5,64.0,40.2,4.180905,46.661298,29.444109,9.094315,9.529033,0,1
313,38.590007,101.0,164.0,36.7,39.8,30.5,116.0,106.4,110.3,64.0,58.2,41.6,4.258235,29.831738,44.134553,13.106154,11.448103,1,0
314,24.974675,81.4,178.5,29.7,32.3,26.6,101.6,79.0,98.5,60.1,54.1,35.9,4.131277,20.457766,38.918782,8.603281,9.156829,1,0
315,46.198494,65.2,171.0,26.2,27.3,22.5,85.0,67.0,95.8,55.9,49.8,37.2,3.691354,24.448794,21.638957,5.948074,7.958757,0,1


## Model

In [6]:
x_columns = data.columns.drop(['masa_piel', 'masa_adiposa', 'masa_muscular', 'masa_residual',
       'masa_osea'])
y_columns = ['masa_piel', 'masa_adiposa', 'masa_muscular', 'masa_residual',
       'masa_osea']
X = data[x_columns].values
y = data[y_columns].values


In [7]:
x_columns

Index(['edad', 'peso', 'talla', 'per_brazo_rel', 'per_brazo_ten',
       'per_antebrazo', 'per_torax', 'per_cintura', 'per_cadera',
       'per_muslo_max', 'per_muslo_medial', 'per_pantorrilla', 'sexo_1',
       'sexo_2'],
      dtype='object')

## split data 

In [52]:
# Create train/test
x_train, x_test, y_train, y_test = train_test_split(    
    X, y, test_size=0.3, random_state=5)

## Model

In [57]:
for index,target in enumerate(y_columns):
    model = Ridge(alpha=0.1)

    model.fit(x_train,y_train[:,index])

    y_pred = model.predict(x_test)
    R2 = r2_score(y_test[:,index],y_pred)
    MAE = mean_absolute_error(y_test[:,index],y_pred)
    MSE = mean_squared_error(y_test[:,index],y_pred)

    print(f'{target}  R2 = {round(R2,3)} and MAE = {round(MAE,3)}')

masa_piel  R2 = 0.997 and MAE = 0.014
masa_adiposa  R2 = 0.738 and MAE = 3.34
masa_muscular  R2 = 0.917 and MAE = 1.781
masa_residual  R2 = 0.926 and MAE = 0.389
masa_osea  R2 = 0.849 and MAE = 0.452
